In [1]:
import pandas as pd
import numpy as np

In [2]:
#Leo el csv de eventos

events_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv", low_memory=False)

## Generacion de features

In [3]:
features_df = pd.DataFrame()

In [4]:
features_df['person'] = events_df.person.unique()

In [5]:
features_df.head()

,person
0,4886f805
1,ad93850f
2,0297fc1e
3,2d681dd8
4,cccea85e


## Cantidad de eventos totales por usuario

In [6]:
df = events_df.groupby('person').event.count()

In [7]:
features_df = features_df.join(df, on='person')

In [8]:
features_df.rename(columns={'event':'cant_eventos_totales'}, inplace=True)

## Cantidad de cada evento por usuario

In [10]:
df = events_df.groupby('person').event.value_counts()

In [11]:
df = df.unstack()
df.head()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
person,,,,,,,,,,,
0008ed71,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0
00091926,15.0,25.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,372.0,34.0
00091a7a,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0
000ba417,1.0,24.0,6.0,1.0,14.0,NaN,1.0,NaN,NaN,153.0,6.0
000c79fe,1.0,NaN,1.0,NaN,1.0,NaN,1.0,9.0,NaN,3.0,1.0


In [12]:
features_df = features_df.join(df, on='person')

In [13]:
features_df.rename(columns={'ad campaign hit':'cant_ach', 'brand listing':'cant_bl', 'checkout':'cant_ch', 
                            'conversion':'cant_cv', 'generic listing':'cant_gl', 'lead':'cant_l',
                           'search engine hit':'cant_seh', 'searched products':'cant_s_p', 
                            'staticpage':'cant_sp', 'viewed product':'cant_vp', 'visited site':'cant_vs'}, inplace=True)

## Labels y separacion de df

In [14]:
labels_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/labels_training_set.csv", low_memory=False)

In [15]:
#Separo en train y to_predict
train_df = features_df.merge(labels_df, on='person', how='right')
train_df.shape

(19414, 14)

In [16]:
to_predict = features_df[~features_df.person.isin(labels_df.person)]
to_predict.shape

(19415, 13)

In [17]:
# Labels: los valores a predecir
#labels = np.array(train_df.label)
labels = train_df.label

#remuevo el label del df
train_df = train_df.drop(columns=['label'])

In [18]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

#Separo en test y train
X_train, X_test, y_train, y_test = \
    train_test_split(train_df.drop(columns=['person']) , labels, test_size=0.2, random_state=123)

In [19]:
X_train.reset_index(inplace=True)
X_train.drop(columns='index', inplace=True)
X_test.reset_index(inplace=True)
X_test.drop(columns='index', inplace=True)
y_train = pd.DataFrame(y_train)

y_train.reset_index(inplace=True)
y_train.drop(columns='index', inplace=True)
y_test = pd.DataFrame(y_test)
y_test.reset_index(inplace=True)
y_test.drop(columns='index', inplace=True)

to_predict = pd.DataFrame(to_predict)
to_predict.reset_index(inplace=True)
to_predict.drop(columns='index', inplace=True)

## XGBoost

In [28]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [29]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))


In [22]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5, 7, 9],
        }

In [30]:
xg_reg = xgb.XGBRegressor(objective ='binary:logistic', booster='dart',
                          base_score=0.1, 
                        colsample_bytree = 0.1, learning_rate = 0.02,
                         alpha = 20, n_estimators = 80)

In [31]:
folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xg_reg, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X_train,y_train), verbose=3, random_state=1001 )

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train, y_train)
timer(start_time) # timing ends here for "start_time" variable

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  15 out of  15 | elapsed:   12.3s finished



 Time taken: 0 hours 0 minutes and 15.05 seconds.


In [32]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)

print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)


 All results:
{'mean_fit_time': array([3.37717819, 2.73430896, 1.91100907, 1.53706209, 2.99855145]), 'std_fit_time': array([0.03069264, 0.04544156, 0.01767216, 0.11123542, 0.13080761]), 'mean_score_time': array([0.04307707, 0.03463252, 0.02538268, 0.02715119, 0.03075449]), 'std_score_time': array([0.00321283, 0.00395736, 0.0022478 , 0.00736802, 0.00381086]), 'param_min_child_weight': masked_array(data=[5, 1, 5, 10, 10],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[9, 7, 4, 3, 9],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=[2, 5, 0.5, 1.5, 1.5],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_colsample_bytree': masked_array(data=[0.6, 0.8, 1.0, 0.8, 0.6],
             mask=[False, False, False, False, False],
       fill_value='?',
 

In [33]:
preds = random_search.predict(X_test)

In [34]:
roc_auc_score(y_test,preds)

0.8147108206838654

## Prediccion

In [37]:
preds_posta = random_search.predict(to_predict.drop(columns=['person']))

In [38]:
preds_posta

array([0.02899743, 0.04775795, 0.03204223, ..., 0.08348841, 0.08348841,
       0.08348841], dtype=float32)

## Genero csv a publicar

In [39]:
to_publish = pd.DataFrame()

In [40]:
to_publish['person'] = to_predict.person
to_publish.shape

(19415, 1)

In [41]:
to_publish['label'] = preds_posta
to_publish.shape

(19415, 2)

In [42]:
to_publish.head()

,person,label
0,4886f805,0.028997
1,0297fc1e,0.047758
2,2d681dd8,0.032042
3,cccea85e,0.047905
4,4c8a8b93,0.061240


In [43]:
to_publish.loc[ to_publish.label > 1 ]

,person,label


In [44]:
to_publish.loc[ to_publish.label < 0 ]

,person,label


In [46]:
to_publish.to_csv('28_11_5.csv', index=False)